Atualizando a biblioteca para plotagem de gráficos

In [10]:
!pip -q install plotly --upgrade

Importing libraries

In [11]:
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Abrindo um arquivo CSV do drive

In [12]:
data = pd.read_csv('./dataset/breast-cancer.csv', sep=';')
data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,03/mai,yes,2,left,right_up,no,recurrence-events
...,...,...,...,...,...,...,...,...,...,...
281,50-59,ge40,30-34,06/ago,yes,2,left,left_low,no,no-recurrence-events
282,50-59,premeno,25-29,03/mai,yes,2,left,left_low,yes,no-recurrence-events
283,30-39,premeno,30-34,06/ago,yes,2,right,right_up,no,no-recurrence-events
284,50-59,premeno,15-19,0-2,no,2,right,left_low,no,no-recurrence-events


Separando os atributos de entrada e de classe

In [13]:
training_data = data.iloc[:,0:9].values # i don't get the target attribute

Target Values - last column

In [14]:
y_classe = data.iloc[:,9].values

| Tratamento de linhas que contenham o '?'

Primeiramente, convertemos essas entradas para "NaN":

In [15]:
import numpy as np
# Substitui Nan por um caracter desejado
data = data.replace('?', np.nan) # presente em dois atributos da base 'breast-cancer'

Apagando as linhas com "NaN"

In [16]:
# remove as linhas com NaN
data = data.dropna() 

Tratamento de dados categóricos (strings)

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
label_encoder = LabelEncoder()

Binarizando atributos nominais

In [19]:
label_encoder_NodeCaps = LabelEncoder()
label_encoder_Breast = LabelEncoder()
label_encoder_Irradiat = LabelEncoder()

In [20]:
training_data[:,4] = label_encoder_NodeCaps.fit_transform(training_data[:,4])
training_data[:,6] = label_encoder_Breast.fit_transform(training_data[:,6])
training_data[:,8] = label_encoder_Irradiat.fit_transform(training_data[:,8])

In [21]:
training_data

array([['40-49', 'premeno', '15-19', ..., 1, 'left_up', 0],
       ['50-59', 'ge40', '15-19', ..., 1, 'central', 0],
       ['50-59', 'ge40', '35-39', ..., 0, 'left_low', 0],
       ...,
       ['30-39', 'premeno', '30-34', ..., 1, 'right_up', 0],
       ['50-59', 'premeno', '15-19', ..., 1, 'left_low', 0],
       ['50-59', 'ge40', '40-44', ..., 0, 'right_up', 0]], dtype=object)

| OneHotEncoder - Vamos **binarizar atributos não ordinais**

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [23]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,1,2,3,5,7])], remainder="passthrough")

In [24]:
training_data = onehotencoder.fit_transform(training_data)

In [25]:
training_data

<286x39 sparse matrix of type '<class 'numpy.float64'>'
	with 2196 stored elements in Compressed Sparse Row format>

Metódo de amostragem **Holdout**

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
# | 80%  |  20%  |
x_treino, x_teste, y_treino, y_teste = train_test_split(training_data, y_classe, test_size = 0.20, random_state = 0)


Pickling is used to store python objects

In [29]:
import pickle

with open('./sample_data/breast_cancer.pkl', mode = 'wb') as f:
    pickle.dump([training_data, x_teste, y_treino, y_teste], f)